In [27]:
import numpy as np
import logging

import torch
import torch.nn.functional as F
import numpy as np


from tqdm import trange
from pytorch_pretrained_bert import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

In [3]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

100%|███████████████████████████████████████████████████████████████████████| 456318/456318 [00:00<00:00, 959259.88B/s]


In [7]:
text_1 = "It was nearly two weeks later when the cosmonaut was jolted out of sleep, "
text_2 = "snug against"
indexed_tokens_1 = tokenizer.encode(text_1)
indexed_tokens_2 = tokenizer.encode(text_2)

# Convert inputs to PyTorch tensors
tokens_tensor_1 = torch.tensor([indexed_tokens_1])
tokens_tensor_2 = torch.tensor([indexed_tokens_2])

In [8]:
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor_1 = tokens_tensor_1.to('cuda')
tokens_tensor_2 = tokens_tensor_2.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    predictions_1, past = model(tokens_tensor_1)
    # past can be used to reuse precomputed hidden state in a subsequent predictions
    # (see beam-search examples in the run_gpt2.py example).
    predictions_2, past = model(tokens_tensor_2, past=past)

# get the predicted last token
predicted_index = torch.argmax(predictions_2[0, -1, :]).item()
predicted_token = tokenizer.decode([predicted_index])

100%|█████████████████████████████████████████████████████████████████████████████| 176/176 [00:00<00:00, 176517.82B/s]


In [9]:
predicted_token

' the'

lol

In [14]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

In [15]:
def top_k_logits(logits, k):
    """
    Masks everything but the k top entries as -infinity (1e10).
    Used to mask logits such that e^-infinity -> 0 won't contribute to the
    sum of the denominator.
    """
    if k == 0:
        return logits
    else:
        values = torch.topk(logits, k)[0]
        batch_mins = values[:, -1].view(-1, 1).expand_as(logits)
        return torch.where(logits < batch_mins, torch.ones_like(logits) * -1e10, logits)

In [28]:
def sample_sequence(model, length, start_token=None, batch_size=None, context=None, temperature=1, top_k=0, device='cuda', sample=True):
    if start_token is None:
        assert context is not None, 'Specify exactly one of start_token and context!'
        context = torch.tensor(context, device=device, dtype=torch.long).unsqueeze(0).repeat(batch_size, 1)
    else:
        assert context is None, 'Specify exactly one of start_token and context!'
        context = torch.full((batch_size, 1), start_token, device=device, dtype=torch.long)
    prev = context
    output = context
    past = None
    with torch.no_grad():
        for i in trange(length):
            logits, past = model(prev, past=past)
            logits = logits[:, -1, :] / temperature
            logits = top_k_logits(logits, k=top_k)
            log_probs = F.softmax(logits, dim=-1)
            if sample:
                prev = torch.multinomial(log_probs, num_samples=1)
            else:
                _, prev = torch.topk(log_probs, k=1, dim=-1)
            output = torch.cat((output, prev), dim=1)
    return output

In [29]:
def run_model(input_text, length=-1, nsamples=1, batch_size=1, temperature=1.0, top_k=0, seed=0):
    
    assert nsamples % batch_size == 0

    np.random.seed(seed)
    torch.random.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    enc = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.to(device)
    model.eval()

    if length == -1:
        length = model.config.n_ctx // 2
    elif length > model.config.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % model.config.n_ctx)

    while True:
        context_tokens = []
        if input_text:
            context_tokens = enc.encode(input_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sample_sequence(
                    model=model, length=length,
                    context=context_tokens,
                    start_token=None,
                    batch_size=batch_size,
                    temperature=temperature, top_k=top_k, device=device
                )
                out = out[:, len(context_tokens):].tolist()
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)
        else:
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sample_sequence(
                    model=model, length=length,
                    context=None,
                    start_token=enc.encoder['<|endoftext|>'],
                    batch_size=batch_size,
                    temperature=temperature, top_k=top_k, device=device
                )
                out = out[:,1:].tolist()
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [30]:
run_model("It was nearly two weeks later when the cosmonaut was jolted out of sleep, jolted by")

07/04/2019 17:51:37 - INFO - pytorch_pretrained_bert.tokenization_gpt2 -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at C:\Users\ethan\.pytorch_pretrained_bert\f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
07/04/2019 17:51:37 - INFO - pytorch_pretrained_bert.tokenization_gpt2 -   loading merges file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at C:\Users\ethan\.pytorch_pretrained_bert\d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
07/04/2019 17:51:38 - INFO - pytorch_pretrained_bert.modeling_gpt2 -   loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin from cache at C:\Users\ethan\.pytorch_pretrained_bert\4295d67f022061768f4adc386234dbdb781c814c39662dd1662221c309962c55.778cf36f5c4e5d94

======================================== SAMPLE 1 ========================================
 a driving lane about two miles across, and crashed in front of a private house, his engine torching and the crew short-circuiting it. Barefoot, limp, fragile and unsteady, he wandered from room to room, and on one occasion, up to his knees, his neck snapped out of shape. He had been dressed, although his trousers were tighter, and attached a turtlelike, unnaturally intelligent nose and heart. He took exception to the omission of his mother and father, who had planned to take him to childbirth if more time did not permit. He wanted to cry. The idea struck a wider range of people – taller people just barely like him, thick people like him early that have attractions he can never even move out of, people of eager sex. Others welcomed a breath of air, (painful strokes on what looked like a tree or a beach, still theirs is stiff, odd). He scooted back into a steering wheel, stared at the plowed mud, 

100%|████████████████████████████████████████████████████████████████████████████████| 512/512 [00:18<00:00, 27.72it/s]


======================================== SAMPLE 1 ========================================
 the explosion that killed his girlfriend Julia, and then more panic erupted, when the bluish-green stinky from Venus ignited and asth rivalled the sheer blue of the sun.

''It devastated the life of the woman, and disabled Zheng even more over here,'' he said.

Analysis An ordinary nine-week job doesn't always suffice for a person like Zheng, he said. This is especially so in China, where about 70 percent of the workforce currently doesn't even speak English. For nice and experienced workers like Zheng, their feet, all, are probably soaked by a glass of water the size of a dung beetle. Chef Yi-Hua Hou, 27, from Chenzhou, who owns Jiangnong City graciously raises prawns for tourists from year to year; the economy hasn't gone strong at all. He addumes large plate of cooked prawns with whipped cream and offers customers a "buy some cheap dinner" discount, but Nik Ek, 35, in Ajincheng, one of Guangz

100%|████████████████████████████████████████████████████████████████████████████████| 512/512 [00:18<00:00, 27.17it/s]


...

======================================== SAMPLE 1 ========================================
 his safety problems and falling into the sea. His situation had not changed for awhile, he denied reports that he was writing a strange symbolic paper on his room, and said that after his sleep, he saw a woman he had barely stepped on before, and he knew her name. Then he rushed into the house to persuade her to give him another name, and that was that. (It wasn't obvious to him that he was inviting another woman to depart suppression orders)

What was less clear was the fact that his experience suggested how far he had fallen behind in travel reported in the NWOC.

FOR A much more detailed view of the bizarre summer in Ussher, near Kuala Lumpur on 4 May 2006, see Contradiction, the most pertinent article in the book Alone in Your Hands: Trails of Fire in its Winter Denim.

The story expressed above, such as the Egyptian calendar with Sun, Moon and Stars, would not be mentioned in Smith's novel.

100%|████████████████████████████████████████████████████████████████████████████████| 512/512 [00:20<00:00, 25.58it/s]


======================================== SAMPLE 1 ========================================
 a steep snowfall nailing the skis with swishing knives, the snow check point was snapped and Cabot questioned by his neighbour to see if the person was breathing. Then while Cabot was in a truck frantically working on the patch of ice, an officer sent his car sideways into the snow, placing the skate prototype on the water's edge, which protected it from harm to Ulysses.

When two police officers arrived at 9:30 looking for a suspect, the lateral effects of the combination of unceasing warm weather forces doctors to interrupt his breathing routine to check for blood. Unlike the Christian Christian Association and Reconstruct Home issued short stocking pulledings and versions of the life jackets with improvised, lockable straps for getting out of the snow, medical tents for large supplies of dried injured tissue next door, remote medical tents in the shed and the four babies living on their own—e

 24%|███████████████████▏                                                            | 123/512 [00:04<00:14, 26.75it/s]


KeyboardInterrupt: 